短文本

In [1]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-

import spacy
import jsonlines
from transformers import AutoTokenizer, AutoModelForCausalLM
from vllm import LLM, SamplingParams
import random
import torch
import os
import numpy as np
import openai
from tqdm import tqdm
import json
import argparse
import ast
import re
from tqdm import tqdm
from collections import Counter
import string
import sys
import time
from utils import PROMPT_DICT, TASK_INST, load_jsonlines, control_tokens, load_special_tokens
from metrics import match, accuracy

import torch, gc

In [2]:
gc.collect()
torch.cuda.empty_cache()


seed = 633

torch.backends.cudnn.deterministic = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)


def postprocess_answer_option_conditioned(answer):
    for token in control_tokens:
        answer = answer.replace(token, "")

    if "</s>" in answer:
        answer = answer.replace("</s>", "")
    if "\n" in answer:
        answer = answer.replace("\n", "")

    if "<|endoftext|>" in answer:
        answer = answer.replace("<|endoftext|>", "")

    return answer


def call_model_rerank_w_scores_batch(prompt, evidences, model, max_new_tokens=15,
                                     ret_tokens=None, rel_tokens=None, grd_tokens=None, ut_tokens=None,
                                     use_seqscore=False, threshold=0.5,
                                     w_rel=1.0, w_sup=1.0, w_use=0.5, mode="adaptive_retrieval", closed=False):
    results = {}
    score_dict = {}
    if mode != "always_retrieve":
        sampling_params = SamplingParams(
            temperature=0.0, top_p=1.0, max_tokens=max_new_tokens, logprobs=32016)
        preds = model.generate([prompt], sampling_params)
        pred_token_ids = preds[0].outputs[0].token_ids
        pred_text = preds[0].outputs[0].text
        pred_log_probs = preds[0].outputs[0].logprobs
        results["no_retrieval"] = pred_text

    # save relevance token scores
    if mode == "always_retrieve":
        do_retrieve = True

    elif mode == "no_retrieval":
        do_retrieve = False

    else:
        for tok, id in ret_tokens.items():
            if id not in pred_log_probs[0]:
                score_dict[tok] = 0
            else:
                prob = pred_log_probs[0][id]
                score_dict[tok] = np.exp(float(prob))
        do_retrieve = score_dict["[Retrieval]"] / (
            score_dict["[Retrieval]"] + score_dict["[No Retrieval]"]) > threshold
    if do_retrieve:
        evidence_augmented_inputs = [prompt + "[Retrieval]<paragraph>{0}\n{1}</paragraph>".format(
            para["title"], para["text"]) for para in evidences]
        sampling_params = SamplingParams(
            temperature=0.0, top_p=1.0, max_tokens=max_new_tokens, logprobs=5000)
        preds = model.generate(evidence_augmented_inputs, sampling_params)

        relevance_score_dict = {}
        grd_score_dict = {}
        ut_score_dict = {}
        overall_scores = {}
        for p_idx, pred in enumerate(preds):
            pred_token_ids = pred.outputs[0].token_ids
            pred_text = pred.outputs[0].text
            pred_log_probs = pred.outputs[0].logprobs
            seq_score = pred.outputs[0].cumulative_logprob / \
                max(len(pred.outputs[0].token_ids), 1)

            relevance_score_dict.setdefault(p_idx, {})
            grd_score_dict.setdefault(p_idx, {})
            ut_score_dict.setdefault(p_idx, {})
            # Compute reward scores
            for tok, id in rel_tokens.items():
                prob = pred_log_probs[0][id] if id in pred_log_probs[0] else -100
                relevance_score_dict[p_idx][tok] = np.exp(float(prob))

            if grd_tokens is not None:
                groundness_token_appear_indices = []
                for tok_idx, tok in enumerate(pred_token_ids):
                    if tok in list(grd_tokens.values()):
                        groundness_token_appear_indices.append(tok_idx)
                        break
                if len(groundness_token_appear_indices) > 0:
                    idx = groundness_token_appear_indices[0]
                    for token, token_id in grd_tokens.items():
                        prob = pred_log_probs[idx][token_id] if token_id in pred_log_probs[idx] else -100
                        grd_score_dict[p_idx][token] = np.exp(float(prob))

            if ut_tokens is not None:
                utility_token_appear_indices = []
                for tok_idx, tok in enumerate(pred_token_ids):
                    if tok in list(ut_tokens.values()):
                        utility_token_appear_indices.append(tok_idx)
                if len(utility_token_appear_indices) > 0:
                    idx = utility_token_appear_indices[0]
                    for token, token_id in ut_tokens.items():
                        prob = pred_log_probs[idx][token_id] if token_id in pred_log_probs[idx] else -100
                        ut_score_dict[p_idx][token] = np.exp(float(prob))

            relevance_score = relevance_score_dict[p_idx]["[Relevant]"] / (
                np.sum(list(relevance_score_dict[p_idx].values())))

            if len(grd_score_dict[p_idx]) == 3:
                gt_sum = np.sum(list(grd_score_dict[p_idx].values()))
                ground_score = (grd_score_dict[p_idx]["[Fully supported]"] / gt_sum) + 0.5 * (
                    grd_score_dict[p_idx]["[Partially supported]"] / gt_sum)
            else:
                ground_score = 0.0

            if len(ut_score_dict[p_idx]) == 5:
                ut_sum = np.sum(list(ut_score_dict[p_idx].values()))
                ut_scores = [-1, -0.5, 0, 0.5, 1]
                utility_score = np.sum(
                    [ut_scores[i] * (ut_score_dict[p_idx]["[Utility:{}]".format(i+1)] / ut_sum) for i in range(len(ut_scores))])
            else:
                utility_score = 0.0

            if use_seqscore is True:
                final_score = np.exp(seq_score) + w_rel * relevance_score + \
                    w_sup * ground_score + w_use * utility_score
            else:
                final_score = w_rel * relevance_score + \
                    w_sup * ground_score + w_use * utility_score

            overall_scores[p_idx] = {"final_score": final_score,
                                     "relevance_score": relevance_score,
                                     "ground_score": ground_score,
                                     "utility_score": utility_score,
                                     "relevance_score_dict": relevance_score_dict,
                                     "grd_score_dict": grd_score_dict,
                                     "ut_score_dict": utility_score}
            score_dict = {"final_score": final_score,
                          "relevance_score": relevance_score,
                          "ground_score": ground_score,
                          "utility_score": utility_score}
            results["retrieval_{}".format(p_idx)] = {
                "pred": pred_text, "score": final_score, "score_dict": score_dict, "ctx": evidences[p_idx]}

    else:
        sampling_params = SamplingParams(
            temperature=0.0, top_p=1.0, max_tokens=max_new_tokens)
        prompt += "[No Retrieval]"
        preds = model.generate([prompt], sampling_params)

        pred = preds[0].outputs[0].text

    # Aggregating answers
    if len(results) == 1:
        postprocessed_pred = postprocess_answer_option_conditioned(pred)
        return postprocessed_pred, results, do_retrieve, score_dict
    else:
        answer2score = {}
        if closed is True:
            for key, result in results.items():
                if key == "no_retrieval":
                    continue
                answer = postprocess_answer_option_conditioned(result["pred"])
                score = result["score"]
                answer2score.setdefault(answer, 0)
                answer2score[answer] += score
            sorted_answers = sorted(
                answer2score.items(), key=lambda x: x[1], reverse=True)
            best_option = sorted_answers[0][0]
        else:
            path2score = {key: item["score"] for key,
                          item in results.items() if key != "no_retrieval"}
            best_path = sorted(path2score.items(),
                               key=lambda x: x[1], reverse=True)[0][0]
            best_option = results[best_path]["pred"]
        return best_option, results, do_retrieve, score_dict


def process_data_evidences(demonstration, top_n):
    ctx_key = "ctxs" if "ctxs" in demonstration else "top_contexts"
    prompt = PROMPT_DICT["prompt_no_input"].format_map(demonstration)
    evidences = demonstration[ctx_key][:top_n]
    return prompt, evidences


def preprocess_input_data(dataset, task=None):
    new_data = []
    if task in TASK_INST:
        instruction = TASK_INST[task]
    else:
        instruction = None
    for item in dataset:
        if task == "arc_c":
            choices = item["choices"]
            answer_labels = {}
            for i in range(len(choices["label"])):
                answer_key = choices["label"][i]
                text = choices["text"][i]
                if answer_key == "1":
                    answer_labels["A"] = text
                if answer_key == "2":
                    answer_labels["B"] = text
                if answer_key == "3":
                    answer_labels["C"] = text
                if answer_key == "4":
                    answer_labels["D"] = text
                if answer_key in ["A", "B", "C", "D"]:
                    answer_labels[answer_key] = text

            if "D" not in answer_labels:
                answer_labels["D"] = ""
            choices = "\nA: {0}\nB: {1}\nC: {2}\nD: {3}".format(
                answer_labels["A"], answer_labels["B"], answer_labels["C"], answer_labels["D"])
            if "E" in answer_labels:
                choices += "\nE: {}".format(answer_labels["E"])
            item["instruction"] = instruction + \
                "\n\n### Input:\n" + item["question"] + choices
            item["answers"] = [item["answerKey"]]
        else:
            prompt = instruction + "\n\n## Input:\n\n" + \
                item["question"] if instruction is not None else item["question"]
            item["instruction"] = prompt
        new_data.append(item)

    return new_data

模型设置

In [3]:
model_name = "self-rag-model"
download_dir = "self-rag-model"
beam_width = 2
world_size = 1
dtype = "half"
device = "cuda"
max_depth = 5
ignore_cont = True

use_grounding = True
use_utility = True
use_seqscore = True

In [4]:
tokenizer = AutoTokenizer.from_pretrained(
        model_name, padding_side="left")

# Get token ids for reflection tokens.
ret_tokens, rel_tokens, grd_tokens, ut_tokens = load_special_tokens(
    tokenizer, use_grounding=use_grounding, use_utility=use_utility)

if world_size is not None:
    model = LLM(model=model_name, download_dir=download_dir,
                dtype=dtype, tensor_parallel_size=world_size,)

else:
    model = LLM(model=model_name,
                download_dir=download_dir, dtype=dtype)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 12-15 14:55:51 config.py:467] Casting torch.bfloat16 to torch.float16.
INFO 12-15 14:55:51 llm_engine.py:73] Initializing an LLM engine with config: model='self-rag-model', tokenizer='self-rag-model', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir='self-rag-model', load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 12-15 14:56:14 llm_engine.py:223] # GPU blocks: 986, # CPU blocks: 512
INFO 12-15 14:56:15 model_runner.py:394] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-15 14:56:21 model_runner.py:437] Graph capturing finished in 6 secs.


In [27]:
# input_file = "../../eval_data/popqa_longtail_w_gs.jsonl"
# input_file = "../../eval_data/test.jsonl"
# output_file = "../../output_data/test"
input_file = "../../eval_data/health_claims_processed.jsonl"
output_file = "../../output_data/health_claims_processed"
task = "fever"
# task = None
max_new_tokens = 50
ndocs = 5
threshold = 0.9
metric = "match"

w_rel = 1
w_sup = 0.5
w_use = 1
# mode = "no_retrieval"
mode = "adaptive_retrieval"
# mode = "always_retrieve"



In [6]:
def main():
    input_path = input_file
    if input_path.endswith(".json"):
        input_data = json.load(open(input_path))
    else:
        input_data = load_jsonlines(input_path)

    input_data = preprocess_input_data(
        input_data, task=task)

    def generate(prompt, evidences, max_new_tokens):
        return call_model_rerank_w_scores_batch(prompt, evidences=evidences, model=model, max_new_tokens=max_new_tokens,
                                                rel_tokens=rel_tokens, ret_tokens=ret_tokens, grd_tokens=grd_tokens, ut_tokens=ut_tokens,
                                                threshold=threshold, use_seqscore=use_seqscore,
                                                w_rel=w_rel, w_sup=w_sup, w_use=w_use, mode=mode, closed=task in ["fever", "arc_c"])

    preds = []
    prompts = []
    golds = []
    metric_results = []
    scores = []
    all_results = []
    count = 1
    tem_metric_results = []
    partitial_right = {}
    for i, row in enumerate(tqdm(input_data)):
        prompt = PROMPT_DICT["prompt_no_input"].format_map(row)
        _, evidences = process_data_evidences(row, top_n=ndocs)
        pred, results, do_retrieve, probs = generate(
            prompt, evidences, max_new_tokens=max_new_tokens)
        all_results.append(results)
        if type(pred) is str and pred[0] == "#" or pred[0] == ":":
            pred = pred[1:]
        prompts.append(prompt)
        preds.append(pred)
        all_results.append(results)
        # i_1 = -1
        # for result in results.keys():
        #     print(f"{i_1}:{results[result]}")
        #     i_1 += 1
        if do_retrieve:
            print(f"当前查询次数{count}")
            count += 1
        if "answers" not in row and "answer" in row:
            row["answers"] = [row["answer"]] if type(
                row["answer"]) is str else row["answer"]
        if metric == "accuracy":
            metric_result = accuracy(pred, row["output"])
                
        elif metric == "match":
            if "SUPPORTS" in pred:
                pred = "true"
            elif "REFUTES" in pred:
                pred = "false"
            metric_result = match(pred, row["answers"])
        else:
            raise NotImplementedError

        if metric_result == 0:
            correct_list = []
            best_index = ""
            for key in results.keys():
                
                if key == "no_retrieval":
                    tmp_pred = results[key]
                else:
                    tmp_pred = results[key]["pred"]
                if tmp_pred == pred:
                    best_index = key
                if metric == "accuracy":
                    tmp_metric_result = accuracy(tmp_pred, row["output"])
                        
                elif metric == "match":
                    if "SUPPORTS" in pred:
                        pred = "true"
                    elif "REFUTES" in pred:
                        pred = "false"
                    tmp_metric_result = match(tmp_pred, row["answers"])
                
                if tmp_metric_result != 0:
                    correct_list.append(key)
            correct_list.append(best_index)
            if len(correct_list) != 0:
                partitial_right[str(i)] = correct_list 
        
        metric_results.append(metric_result)
        tem_metric_results.append(metric_result)
        if i % 10 == 9:
            print("average: {}".format(np.mean(tem_metric_results)))
            tem_metric_results.clear()

    final_results = {"preds": preds, "prompts": prompts, "metric_results": metric_results, "all_results": all_results,
                     "golds": golds,  "metric":  metric, "metric_mean": np.mean(metric_results), "scores": scores}
    with open(output_file, "w") as outfile:
        json.dump(final_results, outfile)

    print("Retrieval Frequencies: {0}".format((count -1) / len(metric_results)))
    return metric_results, all_results,partitial_right

In [28]:
metric_results, all_results, partitial_right = main()

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 14.81it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 16.54it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 16.50it/s]

  0%|          | 3/987 [00:00<03:27,  4.74it/s]

当前查询次数1



Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 16.09it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 16.58it/s]

  1%|          | 5/987 [00:00<03:41,  4.43it/s]

当前查询次数2



Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 16.36it/s]

  1%|          | 6/987 [00:01<04:07,  3.97it/s]

...

当前查询次数532



Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 16.16it/s]

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 16.39it/s]

100%|█████████▉| 986/987 [03:28<00:00,  4.61it/s]

当前查询次数533



Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 16.23it/s]

100%|██████████| 987/987 [03:28<00:00,  4.74it/s]

当前查询次数534
Retrieval Frequencies: 0.541033434650456


In [29]:
print("Final result: {0}".format(np.mean(metric_results)))


Final result: 0.6970618034447822


In [9]:
print(partitial_right)

{'0': ['no_retrieval'], '3': ['no_retrieval'], '4': ['no_retrieval', 'retrieval_4'], '8': ['retrieval_4'], '18': ['retrieval_4'], '19': ['no_retrieval'], '22': ['no_retrieval'], '23': ['no_retrieval'], '26': ['retrieval_4'], '32': ['no_retrieval'], '34': ['no_retrieval', 'retrieval_0', 'retrieval_2', 'retrieval_4'], '36': ['no_retrieval'], '40': ['no_retrieval'], '43': ['no_retrieval'], '47': ['no_retrieval'], '48': ['no_retrieval'], '56': ['no_retrieval'], '57': ['no_retrieval', 'retrieval_0', 'retrieval_1', 'retrieval_4'], '67': ['no_retrieval'], '68': ['no_retrieval'], '69': ['no_retrieval', 'retrieval_4'], '70': ['no_retrieval'], '72': ['no_retrieval'], '73': ['no_retrieval'], '74': ['no_retrieval', 'retrieval_4'], '75': ['retrieval_2', 'retrieval_4'], '80': ['no_retrieval'], '83': ['no_retrieval'], '91': ['no_retrieval', ''], '99': ['retrieval_4'], '100': ['no_retrieval'], '103': ['no_retrieval'], '108': ['retrieval_4'], '109': ['no_retrieval'], '118': ['no_retrieval', 'retrieval_

In [10]:
no_metric_results, no_results, no_partitial_right  = metric_results, all_results, partitial_right

In [11]:
fu_metric_results, fu_results, fu_partitial_right  = metric_results, all_results, partitial_right

In [12]:
ad_metric_results, ad_results, ad_partitial_right  = metric_results, all_results, partitial_right

In [13]:
print(no_metric_results)

[0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 

In [14]:
print(fu_metric_results)

[0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 

In [15]:
print(ad_metric_results)

[0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 

In [16]:
print(ad_partitial_right)

{'0': ['no_retrieval'], '3': ['no_retrieval'], '4': ['no_retrieval', 'retrieval_4'], '8': ['retrieval_4'], '18': ['retrieval_4'], '19': ['no_retrieval'], '22': ['no_retrieval'], '23': ['no_retrieval'], '26': ['retrieval_4'], '32': ['no_retrieval'], '34': ['no_retrieval', 'retrieval_0', 'retrieval_2', 'retrieval_4'], '36': ['no_retrieval'], '40': ['no_retrieval'], '43': ['no_retrieval'], '47': ['no_retrieval'], '48': ['no_retrieval'], '56': ['no_retrieval'], '57': ['no_retrieval', 'retrieval_0', 'retrieval_1', 'retrieval_4'], '67': ['no_retrieval'], '68': ['no_retrieval'], '69': ['no_retrieval', 'retrieval_4'], '70': ['no_retrieval'], '72': ['no_retrieval'], '73': ['no_retrieval'], '74': ['no_retrieval', 'retrieval_4'], '75': ['retrieval_2', 'retrieval_4'], '80': ['no_retrieval'], '83': ['no_retrieval'], '91': ['no_retrieval', ''], '99': ['retrieval_4'], '100': ['no_retrieval'], '103': ['no_retrieval'], '108': ['retrieval_4'], '109': ['no_retrieval'], '118': ['no_retrieval', 'retrieval_

In [17]:
print(fu_partitial_right)

{'0': ['no_retrieval'], '3': ['no_retrieval'], '4': ['no_retrieval', 'retrieval_4'], '8': ['retrieval_4'], '18': ['retrieval_4'], '19': ['no_retrieval'], '22': ['no_retrieval'], '23': ['no_retrieval'], '26': ['retrieval_4'], '32': ['no_retrieval'], '34': ['no_retrieval', 'retrieval_0', 'retrieval_2', 'retrieval_4'], '36': ['no_retrieval'], '40': ['no_retrieval'], '43': ['no_retrieval'], '47': ['no_retrieval'], '48': ['no_retrieval'], '56': ['no_retrieval'], '57': ['no_retrieval', 'retrieval_0', 'retrieval_1', 'retrieval_4'], '67': ['no_retrieval'], '68': ['no_retrieval'], '69': ['no_retrieval', 'retrieval_4'], '70': ['no_retrieval'], '72': ['no_retrieval'], '73': ['no_retrieval'], '74': ['no_retrieval', 'retrieval_4'], '75': ['retrieval_2', 'retrieval_4'], '80': ['no_retrieval'], '83': ['no_retrieval'], '91': ['no_retrieval', ''], '99': ['retrieval_4'], '100': ['no_retrieval'], '103': ['no_retrieval'], '108': ['retrieval_4'], '109': ['no_retrieval'], '118': ['no_retrieval', 'retrieval_

In [18]:
# 部分出错280项(其中查询但是没有选对的有262项)


In [19]:
num = 0
for key,value in ad_partitial_right.items():
    if len(value) > 1 and value[0] != "no_retrieval":
        num += 1
print(num)

9


In [20]:
len(fu_partitial_right)

299

In [21]:
length = len(no_metric_results)
num = 0
for i in range(length):
    if ad_metric_results[i] == 0 and fu_metric_results[i] == 1:
        num += 1
print(num)

0


In [22]:
length = len(no_metric_results)
num = 0
for i in range(length):
    if ad_metric_results[i] == 0 and no_metric_results[i] == 1:
        num += 1
        print(i)
        break
print(num)

0


In [23]:
print(ad_results)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [24]:
import math
true_num = [0 for _ in range(10)]
for prob in probs_true_list:
    true_num[math.floor(prob*10)] += 1
false_num = [0 for _ in range(10)]
for prob in probs_false_list:
    false_num[math.floor(prob*10)] += 1

NameError: name 'probs_true_list' is not defined

In [ ]:
print(false_num)
print(true_num)